In [2]:
using Flux

# Define the binarize function
binarize(x) = x .> 0.5

# Define the model architecture
model = Chain(
  Dense(10000, 100),
  binarize,
  Dense(100, 10),
  softmax
)

Chain(
  Dense(10000 => 100),                  # 1_000_100 parameters
  binarize,
  Dense(100 => 10),                     # 1_010 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 1_001_110 parameters, 3.819 MiB.

In [11]:
# Define the loss function and optimizer
loss(x, y) = Flux.crossentropy(model(x), y)
optim = ADAM()

# Generate some dummy data for testing
x = rand((0,1), 10000)
y = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

# Train the model
for i = 1:1000
  Flux.train!(loss, Flux.params(model), [(x, y)], optim)
end

# Test the model
test_x = rand(Float32, 10000)
prediction = argmax(model(test_x))


6

In [20]:
using CSV
using DataFrames
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
#Load embeddings from last hidden layer of ESM-2 model (21x1280)
aa_embeddings = DataFrame(CSV.File("../data/amino_acid_embeddings.csv"))
amino_acids_esm = aa_embeddings.protein_ID
aa_emb = Matrix(aa_embeddings[:, 2:end])

HDV_mat_bit = nested_arrays2mat([bithdv() for i in 1:size(aa_emb)[2]], true)
AA_bit_esm = permutedims(mat_scaler(aa_emb * HDV_mat_bit, 0, 1, 2) .|> round)

x = [AA_bit_esm[:, i] for i in 1:21]
y = amino_acids_esm


# Define the loss function and optimizer
loss(x, y) = Flux.crossentropy(model(x), y)
optim = ADAM()

# Train the model
for i = 1:1000
    for i in 1:21
        Flux.train!(loss, Flux.params(model), [(x[i], y[i])], optim)
    end
end

# Test the model
test_x = amino_acids_esm
prediction = argmax(model(test_x))


MethodError: MethodError: no method matching *(::String1, ::Float32)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:591
  *(!Matched::T, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:385
  *(::Union{AbstractChar, AbstractString}, !Matched::Union{AbstractChar, AbstractString}...) at strings/basic.jl:260
  ...